# Line detection

La détection de ligne consiste à détecter des alignements de points sur une image de contours.

```{figure} figs/line-example.png
---
width: 250px
name: F:feature:lines:example
```

La méthode la plus courante pour cela est la transformée de Hough [Hough 1962].
Comme la transformée de Fourier, elle transpose l'image du domaine spatial vers un autre domaine,
où les informations d'intérêt sont représentées différemment.
En l'occurrence, les lignes dans le domaine spatial sont transformées en points dans le domaine de Hough.

La transformée de Hough n'est pas restreinte aux lignes et peut être utilisée pour détecter une autre forme,
tant qu'il existe une paramétrisation mathématique de cette forme.
Le principe de fonctionnement reste cependant le même, et la suite de ce chapitre se concentre sur la détection de lignes.

## Première paramétrisation

Dans le domaine spatial $(x,y)$, une ligne est paramétrée par son coefficient directeur $a$ et son ordonnée à l'origine $b$ :

$$
y = a x + b.
$$

Ainsi, toute droite peut être représentée par le couple $(a,b)$.
C'est l'idée de Hough : chaque ligne de l'image sera représentée par un point dans le domaine de Hough $(a,b)$.
Le domaine de Hough est aussi appelé espace des paramètres.

```{figure} figs/feat-1.png
---
width: 450px
name: F:feature:lines:hough-1
---
La transformée de Hough transforme une droite de l'image en un point dans l'espace des paramètres.
```

À l'inverse, un point de l'image est représenté par une droite dans l'espace des paramètres
(qui a pour équation $b = \alpha a + \beta$).

```{figure} figs/feat-2.png
---
width: 450px
name: F:feature:lines:hough-2
```

En particulier, une droite constante $b=\beta$ dans l'espace des paramètres correspond à un point d'abscisse $x=0$ dans l'image.

```{figure} figs/feat-3.png
---
width: 450px
name: F:feature:lines:hough-3
```

Enfin, lorsque plusieurs points dans l'image sont alignés, leurs droites respectives dans l'espace des paramètres
se coupent en un unique point, qui définit l'équation de la droite les reliant.

```{figure} figs/feat-5.png
---
width: 450px
name: F:feature:lines:hough-5
```

Cependant, comme l'espace des paramètres est fini et discrétisé (puisqu'il est traité par un ordinateur),
certaines lignes de l'image ne pourront pas y être représentées.
C'est le cas par exemple d'une droite verticale pour laquelle $a=\infty$.
Aussi, une autre paramétrisation des droites est utilisées dans la pratique.

## Nouvelle paramétrisation

Pour éviter le problème de l'espace de paramétrisation $(a,b)$, les droites sont définies à partir de deux autres paramètres :
* la longueur $s$ du segment reliant l'origine avec le point le plus proche de la droite (ce segment est donc perpendiculaire à la droite),
* l'angle $\theta$ que fait ce segment avec l'axe des abscisses.

Chaque ligne de l'image est donc paramétrée par le couple $(\theta,s)$ qui correspond à un point dans l'espace de paramètre $(\theta,s)$.

```{figure} figs/feature-50.png
---
width: 450px
name: F:feature:lines:houghnew-1
---
Nouvelle paramétrisation de la transformée de Hough.
```

<!-- Equation de la droite paramétrée ainsi -->

On peut montrer que pour chaque point $(x_i,y_i)$ de l'image, une sinusoïde d'équation $s = x_i \cos(\theta) + y_i \sin(\theta)$
est associée dans l'espace $(\theta,s)$.
Les sinusoïdes correspondant aux points d'une même droite se coupent au point $(s^*,\theta^*)$ paramétrisant cette droite.

L'algorithme de la transformée de Hough est le suivant :

```{figure} figs/feat-9.png
---
width: 450px
name: F:feature:lines:houghnew-algo
```

La {numref}`F:feature:lines:example` donne un exemple de transformée de Hough sur une image représentant un carré
(on peut imaginer que cette image provient d'une détection de contour).

<!-- dans l'example, afficher d'abord l'image avant détection de contour (un losange blanc donc) -->

```{figure} figs/feat-10.png
---
width: 600px
name: F:feature:lines:houghnew-example
---
Tranformée de Hough associée à l'image de gauche.
Les sinusoïdes se coupent en quatre points (très lumineux) telle que l'association entre les lignes de l'image et les points sont :
A$\rightarrow$2, B$\rightarrow$3, C$\rightarrow$1 et D$\rightarrow$4;
```

<!-- Autre exemple : Léo Letouzey + Thomas Chabrier 2008 -->

Finalement, la transformée de Hough présente plusieurs avantages :
* elle est robuste au bruit,
* elle est robuste aux occlusions (elle peut détecter des objets partiellement recouverts)

De plus, elle est extensible à tout objet paramétré (cercles, ellipses...).
Par exemple, un cercle est paramétré par trois paramètres (les coordonnées du centre et le rayon).
Une transformée de Hough est donc possible, et son espace de paramètres est en trois dimensions.

On voit donc l'inconvénient principal de cette méthode,
c'est-à-dire que la dimension de l'accumulateur est égale aux nombres de paramètres,
et donc le temps de calcul et la mémoire utilisée deviennent vite conséquents.